<a href="https://colab.research.google.com/github/git-commit/refactored-octo-spork/blob/main/d7_1_slice_cnn_vgg_(centralized%2C_sweep_0%2C_coordinator).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Centralized Learning experiment with D7.1

## Configuration

In [ ]:
project_name = "Thesis d7.1 (production)"
num_clients = 1

config = {"num_clients": num_clients,
        "client_lr":0.0001,
        "lr_decay": 0.05,
        "epochs": 150,
        "batch_size":64,
        "loss_function": "categorical_crossentropy",
        "model": "VGG_Simple",
        "dataset": "HUST-19 (slices)",
        "dataset_size_limit": None,
        "sampling": None if num_clients < 2 else "stratified_shuffle_deterministic",
        "random_seed": 4242
      }

debug_run = False
if debug_run:
  config["epochs"] = 1
  config["num_rounds"] = 1
  config["dataset_size_limit"] = config["batch_size"]*2

## Download And Verify Dataset

In [ ]:
# Original Dataset on: http://ictcf.biocuckoo.cn
#NiCT: http://ictcf.biocuckoo.cn/patient/CT/NiCT.zip
#pCT: http://ictcf.biocuckoo.cn/patient/CT/pCT.zip
#nCT: http://ictcf.biocuckoo.cn/patient/CT/nCT.zip
#nCT(no disease): http://ictcf.biocuckoo.cn/patient/CT/nCT%20(No%20disease).zip

![ ! -f pCT.tar ] && wget -qO- https://geile.software/projects/thesis/d7.1/pCT.tar | tar x
![ ! -f nCT.tar ] && wget -qO- https://geile.software/projects/thesis/d7.1/nCT.tar | tar x
![ ! -f NiCT.tar ] && wget -qO- https://geile.software/projects/thesis/d7.1/NiCT.tar | tar x
![ ! -f checksums ] && wget -q https://geile.software/projects/thesis/d7.1/checksums

# create reproducible tars: http://h2.jaguarpaw.co.uk/posts/reproducible-tar/
![ ! -f pCT.tar ] && tar --sort=name --mtime='2015-10-21 00:00Z' --owner=0 --group=0 --numeric-owner --format=gnu --mode="go-rwx,u-rw" -cf pCT.tar pCT/
![ ! -f nCT.tar ] && tar --sort=name --mtime='2015-10-21 00:00Z' --owner=0 --group=0 --numeric-owner --format=gnu --mode="go-rwx,u-rw" -cf nCT.tar nCT/
![ ! -f NiCT.tar ] && tar --sort=name --mtime='2015-10-21 00:00Z' --owner=0 --group=0 --numeric-owner --format=gnu --mode="go-rwx,u-rw" -cf NiCT.tar NiCT/

import os
if os.system('cat checksums | sha256sum --check --status') != 0:
  raise Exception("Download Checksums wrong")
else:
  print("Checksums correct")

Exception: ignored

## Dependencies for d7.1
To use Inception Net V3 and ChexNet model in your application developments, you must have installed the following dependencies:

Python 3.7

OpenCV-python 3.4.2

Scikit-image 0.15.0

Scikit-learn 0.21.2

Tensorflow 1.13.1

Keras 2.2.4

In [ ]:
![ -f pip-setup-finished ] && echo "dependencies already set up on this runtime"
![ ! -f pip-setup-finished ] && pip --version
![ ! -f pip-setup-finished ] && pip uninstall -y tensorflow tensorflow-gpu opencv-python scikit-image scikit-learn keras
![ ! -f pip-setup-finished ] && pip install tensorflow-gpu==1.13.1
![ ! -f pip-setup-finished ] && pip install keras==2.2.4
![ ! -f pip-setup-finished ] && pip install opencv-python==3.4.2.17
![ ! -f pip-setup-finished ] && pip install scikit-image==0.15.0
![ ! -f pip-setup-finished ] && pip install scikit-learn==0.22

![ ! -f pip-setup-finished ] && pip install wandb

!touch pip-setup-finished


In [ ]:
import wandb

In [ ]:
import os
import cv2 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import ChainMap
from keras.utils.np_utils import to_categorical
from keras.models import load_model  
from keras.utils import plot_model
from keras import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Softmax,Activation,Dense,Dropout
from keras.callbacks import Callback,ModelCheckpoint
from sklearn.linear_model import LogisticRegressionCV 
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from keras import optimizers

from keras.applications.vgg16 import VGG16
from keras.layers import Input
import keras
import sklearn

In [ ]:
# Set random seeds
import numpy as np
import tensorflow as tf
import random

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.
np.random.seed(config["random_seed"])

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.
random.seed(config["random_seed"])

# The below set_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see:
# https://www.tensorflow.org/api_docs/python/tf/random/set_seed
tf.random.set_random_seed(config["random_seed"])

os.environ['PYTHONHASHSEED'] = str(config["random_seed"])

## Utility Functions

In [ ]:
def corp_margin(img2):
    img2=np.asarray(img2)
    (row, col) = img2.shape
    row_top = 0
    raw_down = 0
    col_top = 0
    col_down = 0
    axis1=img2.sum(axis=1)
    axis0=img2.sum(axis=0)
    for r in range(0, row):
        if axis1[r] > 30:
            row_top = r
            break
    for r in range(row - 1, 0, -1):
        if axis1[r] > 30:
            raw_down = r
            break
    for c in range(0, col):
        if axis0[c] > 30:
            col_top = c
            break
    for c in range(col - 1, 0, -1):
        if axis0[c] > 30:
            col_down = c
            break
    a=raw_down+ 1 - row_top-(col_down+ 1-col_top)
    if a>0:
            w=raw_down+ 1-row_top
            col_down=int((col_top+col_down + 1)/2+w/2)
            col_top = col_down-w
            if col_top < 0:
                col_top = 0
                col_down = col_top + w
            elif col_down >= col:
                col_down = col - 1
                col_top = col_down - w
    else:
            w=col_down + 1- col_top
            raw_down = int((row_top + raw_down + 1) / 2 + w/2)
            row_top =  raw_down-w
            if row_top < 0:
                row_top = 0
                raw_down = row_top + w
            elif raw_down >= row:
                raw_down = row - 1
                row_top = raw_down - w
    if row_top==raw_down:
        row_top=0
        raw_down=99
        col_top = 0
        col_down = 99
    new_img = img2[row_top:raw_down + 1, col_top:col_down + 1]
    return new_img


def read_ct_img_bydir(target_dir):
    img=cv2.imdecode(np.fromfile(target_dir,dtype=np.uint8),cv2.IMREAD_GRAYSCALE)
    img = corp_margin(img)
    img=cv2.resize(img,(200,200))
    return img

from tensorflow.keras.callbacks import Callback,ModelCheckpoint
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=()):
        super(Callback, self).__init__()
        self.x_val,self.y_val = validation_data
    def on_epoch_end(self, epoch, log={}):
        y_pred = self.model.predict(self.x_val)
        AUC1 = roc_auc_score(self.y_val[:,0], y_pred[:,0])
        AUC2 = roc_auc_score(self.y_val[:,1], y_pred[:,1])
        AUC3 = roc_auc_score(self.y_val[:,2], y_pred[:,2])
        print('val_AUC NiCT epoch:%d: %.6f' % (epoch+1, AUC1))
        print('val_AUC pCT epoch:%d: %.6f' % (epoch+1, AUC2))
        print('val_AUC nCT epoch:%d: %.6f' % (epoch+1, AUC3))
        print(f'val_AUC_avg={(AUC1+AUC2+AUC3)/3}')

        wandb.log({"val_auc_nict": AUC1,
                   "val_auc_pct": AUC2,
                   "val_auc_nct": AUC3}, commit=False)

## Model Setup


In [ ]:
def VGG_Simple():
    model=Sequential()
    model.add(Conv2D(64,(3,3),strides=(1,1),input_shape=(200,200,1),padding='same',activation='relu',kernel_initializer='uniform'))
    model.add(Conv2D(64,(3,3),strides=(1,1),padding='same',kernel_initializer='uniform',activation='relu'))
    model.add(MaxPooling2D((2,2)))
    model.add(Conv2D(32,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
    model.add(Conv2D(32,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
    model.add(MaxPooling2D((2,2)))
    model.add(Conv2D(16,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
    model.add(Conv2D(16,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
    model.add(MaxPooling2D((2,2)))
    model.add(Flatten())
    model.add(Dense(64,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3,activation='softmax'))
    return model

def VGG_16():
    input_tensor = Input(shape=(200,200,1))

    weight_model = VGG16(weights='imagenet', include_top=False) #Load ResNet50V2 ImageNet pre-trained weights
    weight_model.save_weights('weights.h5') #Save the weights
    base_model = VGG16(weights=None, include_top=False, input_tensor=input_tensor) #Load the ResNet50V2 model without weights
    base_model.load_weights('weights.h5',skip_mismatch=True, by_name=True) #Load the ImageNet weights on the ResNet50V2 model except the first layer(because the first layer has one channel in our case)

    model=Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(3,activation='softmax'))
    return model


## Data Loading

In [ ]:
target_dir1='NiCT/'
target_dir2='pCT/'
target_dir3='nCT/'
target_list1=[target_dir1+file for file in os.listdir(target_dir1)]
target_list2=[target_dir2+file for file in os.listdir(target_dir2)]
target_list3=[target_dir3+file for file in os.listdir(target_dir3)]

if config["dataset_size_limit"] != None:
  target_list1 = random.sample(target_list1, config["dataset_size_limit"])
  target_list2 = random.sample(target_list2, config["dataset_size_limit"])
  target_list3 = random.sample(target_list3, config["dataset_size_limit"])

target_list=target_list1+target_list2+target_list3
print(f"len(target_list)={len(target_list)}")
y_list=to_categorical(np.concatenate(np.array([[0]*len(target_list1),
                                               [1]*len(target_list2),
                                               [2]*len(target_list3)])),3)
X=np.array([read_ct_img_bydir(file) for file in target_list])[:,:,:,np.newaxis]
X_train, X_val, y_train, y_val = train_test_split(X, y_list, test_size=0.1, stratify=y_list, random_state=config["random_seed"])

print(f"len(X_train)={len(X_train)}")

In [ ]:
import math
import random
import collections

def get_two_client_indices_stratified_shuffle_deterministic(y, random_seed):
  splitter = sklearn.model_selection.StratifiedShuffleSplit(n_splits=1, train_size = 0.5, random_state=random_seed)
  x = np.zeros(len(y))
  split = list(splitter.split(x,y))
  return split[0]

def get_client_indices_stratified_shuffle_deterministic(num_clients, y, random_seed):
  labels = list(y)
  overall_indices = list(range(len(y)))

  picked_ids_for_client = []

  for i in range(num_clients-1):
    remaining_splits = num_clients - i
    split_percentage = 1 / remaining_splits

    splitter = sklearn.model_selection.StratifiedShuffleSplit(n_splits=1, train_size = split_percentage, random_state=random_seed+i)
    split = list(splitter.split(overall_indices, labels))
    picked_ids = split[0][0]
    
    indices_in_overall = [overall_indices[i] for i in picked_ids]
    picked_ids_for_client.append(indices_in_overall)

    print(f"starting with: {overall_indices}")
    # remove picked_ids from overall_indices as well as labels
    for i in sorted(picked_ids, reverse=True):
      del overall_indices[i]
      del labels[i]

    print(f"picked indices: {picked_ids}\nremaining: {overall_indices}\n")

  picked_ids_for_client.append(overall_indices)
  return picked_ids_for_client

if config["num_clients"] > 1:
  ind = get_client_indices_stratified_shuffle_deterministic(y_train, config["random_seed"])[0]
  config["dataset_size_limit"] = len(ind)

  print(f"len(X_train)={len(X_train)}\nLimiting to {config['dataset_size_limit']} samples with indices: {ind}")
  X_train = np.array([X_train[i] for i in ind])
  y_train = np.array([y_train[i] for i in ind])
  print(f"len(X_train)={len(X_train)}")

In [ ]:
class MetricsEvaluation(tf.keras.callbacks.Callback):
    def __init__(self, validation_data=()):
        super(Callback, self).__init__()
        self.x_val,self.y_val = validation_data
        self.best_val_loss = float("inf")

    def on_epoch_end(self, epoch, log={}):
        '''
        We gather the following metrics (per class and as avg):
        - accuracy
        - recall/sensitivity
        - specificity
        - precision (positive predictive value)
        - f1-score

        [1] https://onlinelibrary.wiley.com/doi/full/10.1111/j.1651-2227.2006.00180.x
        '''
        epoch = epoch + 1

        predicted_class = self.model.predict_classes(self.x_val)
        true_class = np.argmax(self.y_val, axis=1)
        label_names = ["nict", "pct", "nct"]
        report = compute_metrics(true_class, predicted_class, label_names)
        log["epoch"] = epoch
        log["val_metrics"] = report
        del log["val_acc"]
        print(f"epoch {epoch} log={log}")
        wandb.log(log)

        # Summarize if best epoch
        if log["val_loss"] < self.best_val_loss:
          print(f"val_loss improved from {self.best_val_loss} to {log['val_loss']} in epoch {epoch}")
          self.best_val_loss = log["val_loss"]

          wandb.summary["best_epoch"] = epoch
          wandb.summary["val_loss"] = self.best_val_loss
          wandb.summary["best_val_confusion_matrix"] = sklearn.metrics.confusion_matrix(true_class, predicted_class)
          wandb.summary["val_metrics"] = report

          self.best_report = report

    def on_federated_round_end(self, round_num, state, metrics):
        print(f'round {round_num}, metrics={metrics}')

        state.model.assign_weights_to(self.model)
        val_loss, val_acc = self.model.evaluate(X_val, y_val, verbose=0)
        
        # Epoch-1 because we add 1 to the callback in on_epoch_end and count them starting at 1 instead of 0
        epoch = (round_num * config.epochs_per_client) - 1
        self.on_epoch_end(epoch, log={"round": round_num,
                                      "acc": metrics["train"]["categorical_accuracy"],
                                      "loss": metrics["train"]["loss"],
                                      "val_acc": val_acc,
                                      "val_loss": val_loss})
    def submit_best(self):      
      wandb.summary["val_loss"] = self.best_val_loss
      wandb.summary["val_metrics"] = self.best_report

def acc(confusion_matrix):
  tn, fp, fn, tp = confusion_matrix.ravel()
  return (tp+tn)/(tn+fp+fn+tp)

def per_class_accuracy(ytrue, ypred):
  mcm = sklearn.metrics.multilabel_confusion_matrix(ytrue, ypred)
  return [acc(cm) for cm in mcm]

def sp(confusion_matrix):
  tn, fp, fn, tp = confusion_matrix.ravel()
  return (tn)/(tn+fp)

def per_class_specificity(ytrue, ypred):
  mcm = sklearn.metrics.multilabel_confusion_matrix(ytrue, ypred)
  return [sp(cm) for cm in mcm]

def compute_metrics(true_class, predicted_class, label_names):
  report = sklearn.metrics.classification_report(true_class, predicted_class, target_names = label_names, output_dict=True)

  accs = per_class_accuracy(true_class, predicted_class)
  specs = per_class_specificity(true_class,predicted_class)
  for i in range(3):
    report[label_names[i]]["accuracy"] = accs[i]
    report[label_names[i]]["specificity"] = specs[i]

  print(sklearn.metrics.classification_report(true_class, predicted_class, target_names = label_names))
  return report

In [ ]:
from keras import backend as K
def train():
  K.clear_session()
  run = wandb.init(config=config)
  print(run.config)
  run.name=f"{{model: \"{run.config.model.lower()}\", client_lr: {run.config.client_lr}, lr_decay: {run.config.lr_decay}, num_clients: {run.config.num_clients}}}"
  checkpoint = ModelCheckpoint(os.path.join(wandb.run.dir, "model.h5"), save_weights_only = False, monitor='val_loss', verbose=1, save_best_only=True, mode='auto', period=1)
  RocAuc = RocAucEvaluation(validation_data=(X_val,y_val))
  AdditionalMetrics = MetricsEvaluation(validation_data=(X_val,y_val))

  model = None
  if run.config.model == "VGG_16":
      model = VGG_16()
  else:
      model = VGG_Simple()

  model.compile(optimizer=optimizers.Adam(lr=run.config.client_lr, decay=run.config.lr_decay), loss= run.config.loss_function,metrics=['accuracy'])
  model.summary()
  wandb.summary['params'] = model.count_params()
  wandb.summary['len_train'] = len(X_train)

  metrics_calculator = MetricsEvaluation((X_val, y_val))
  metrics_calculator.model = model

  # round 0 will be the freshly initialized model with no training
  loss, acc = model.evaluate(X_train, y_train, verbose=0)
  val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
  metrics_calculator.on_epoch_end(-1, log={"acc": acc,"loss": loss,"val_acc": val_acc, "val_loss": val_loss})

  # train other rounds
  history = model.fit(X_train, y_train, epochs=run.config.epochs, batch_size=run.config.batch_size, class_weight = 'auto', validation_data=(X_val, y_val), callbacks=[checkpoint,RocAuc,AdditionalMetrics],verbose=1)
  metrics_calculator.submit_best()

In [ ]:
sweep_config = {
  "name": "Sweep Centralized 1",
  "method": "grid",
  "parameters": {
        "client_lr": {
            "values": [1e-2, 1e-3, 1e-4]
        },
        "model": {
            "values": ["VGG_16", "VGG_Simple"]
        },
        "lr_decay": {
            "values": [0, 0.05]
        },
        "num_clients": {
            "values": [3]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project=project_name)
wandb.agent(sweep_id, function=train)